# Задание

Обучить модель GPT-2 (любую из подмножества моделей этой архитектуры) суммаризации на основе данных и предобученной модели из открытого доступа на русском языке. Нельзя брать датасет, если выбранная модель на нём обучалась.

- Выбрать модель: https://huggingface.co/models
- Выбрать данные: https://huggingface.co/datasets

В качестве результата должны быть представлены примеры суммаризации из тестовой части выбранного набора данных для исходной и дообученой модели.

In [1]:
import torch
from torch import optim
from torch.utils.data import DataLoader
from torchmetrics.functional.text import bleu_score
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import lightning.pytorch as pl
from datasets import load_dataset
from lightning.pytorch.loggers import TensorBoardLogger
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
os.environ['http_proxy'] = 'http://proxy.stc:3128'
os.environ['https_proxy'] = 'http://proxy.stc:3128'
os.environ['ftp_proxy'] = 'http://proxy.stc:3128'

In [4]:
class GenModel(pl.LightningModule):
    def __init__(self, model_name_or_path):
        super().__init__()
        
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, truncation_side="left", padding_side='left')
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.model = AutoModelForCausalLM.from_pretrained(model_name_or_path, pad_token_id=self.tokenizer.eos_token_id)
    
    def training_step(self, batch, batch_idx):
        
        loss = self.model(input_ids=batch["input_ids"].to(device="cuda"), attention_mask=batch["attention_mask"].to(device="cuda"), labels=batch["input_ids"].to(device="cuda")).loss
        self.log("train_loss", loss.item(), prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        val_loss = self.model(input_ids=batch["input_ids"].to(device="cuda"), attention_mask=batch["attention_mask"].to(device="cuda"), labels=batch["input_ids"].to(device="cuda")).loss
        self.log("val_loss", val_loss.item(), prog_bar=True)
        return val_loss
    
    def test_step(self, batch, batch_idx):
        outputs = self.model.generate(batch["input_ids"].to(device="cuda"), attention_mask=batch["attention_mask"].to(device="cuda"), max_new_tokens=200)
        #outputs = self.model.generate(batch["input_ids"], attention_mask=batch["attention_mask"])
        preds = []
        for i in range(len(outputs)):
            pred = self.tokenizer.decode(outputs[i], skip_special_tokens=True)
            preds.append(pred)
        
        bleu = bleu_score(preds, batch["summaries"], n_gram=1)
        self.log("bleu", bleu.item(), prog_bar=True)
        #preds = np.array(preds)
        
        return bleu#, preds
    
    def predict_step(self, batch, batch_idx):
        outputs = self.model.generate(batch["input_ids"], attention_mask=batch["attention_mask"], max_new_tokens=200)
        
        preds = []
        for i in range(len(outputs)):
            pred = self.tokenizer.decode(outputs[i], skip_special_tokens=True)
            preds.append(pred)
        
        return preds
    
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=2e-5)
        return optimizer

In [5]:
class GenDataModule(pl.LightningDataModule):
    def __init__(self, model_name_or_path, ds_name):
        super().__init__()
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, truncation_side="left", padding_side='left')
        self.tokenizer.pad_token = self.tokenizer.eos_token
        #self.model = AutoModelForCausalLM.from_pretrained(model_name_or_path, pad_token_id=self.tokenizer.eos_token_id)
        self.ds_name = ds_name
        
    def prepare_data(self):
        self.ds = load_dataset(self.ds_name)
        for split in self.ds:
            #self.ds[split] = self.ds[split].select(list(range(5000)))
            if split == "train":
                 self.ds[split] = self.ds[split].select(list(range(5000)))
            else:
                self.ds[split] = self.ds[split].select(list(range(100)))
            if split == "test":
                self.ds[split] = self.ds[split].map(self.test_tokenize)
            else:
                self.ds[split] = self.ds[split].map(self.tokenize)
        
    def train_dataloader(self):
        train_split = self.ds["train"]
        return DataLoader(train_split, batch_size=4, collate_fn=self.collate)
    
    def val_dataloader(self):
        val_split = self.ds["validation"]
        return DataLoader(val_split, batch_size=4, collate_fn=self.collate)
    
    def test_dataloader(self):
        test_split = self.ds["test"]
        return DataLoader(test_split, batch_size=4, collate_fn=self.collate_test)
    
    def predict_dataloader(self):
        pred_split = self.ds["test"]
        return DataLoader(pred_split, batch_size=4, collate_fn=self.collate_test)
    
    def tokenize(self, row):
        row = self.tokenizer(f'{row["text"]} summary: {row["summary"]}', max_length=512, padding='max_length', truncation=True, return_tensors="pt")
        return row
    
    def test_tokenize(self, row):
        row = self.tokenizer(f'{row["text"]} summary:', max_length=512, padding='max_length', truncation=True, return_tensors="pt")
        return row
    
    def collate(self, samples):
        input_ids = torch.stack([torch.tensor(s["input_ids"]).squeeze() for s in samples])
        attention_mask = torch.stack([torch.tensor(s["attention_mask"]).squeeze() for s in samples])
        
        return {"input_ids": input_ids, "attention_mask": attention_mask}
    
    def collate_test(self, samples):
        input_ids = torch.stack([torch.tensor(s["input_ids"]).squeeze() for s in samples])
        attention_mask = torch.stack([torch.tensor(s["attention_mask"]).squeeze() for s in samples])
        summaries = [row["summary"] for row in samples]
        
        return {"input_ids": input_ids, "attention_mask": attention_mask, "summaries": summaries}
        

In [6]:
gen_model = GenModel("ai-forever/rugpt3small_based_on_gpt2")

In [7]:
gen_dm = GenDataModule("ai-forever/rugpt3small_based_on_gpt2", "IlyaGusev/gazeta")

In [21]:
trainer = pl.Trainer(max_epochs=10)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [9]:
pred = trainer.predict(model=gen_model, datamodule=gen_dm)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |                                             | 0/? [00:00<?, ?it/s]

In [12]:
pred[0][0]

' Берлине прошли массовые акции протеста против антикоронавирусных мер: ношения масок и соблюдения безопасного расстояния в 1,5 м друг от друга. Манифестанты требуют ослабить или отменить ограничения. По данным таблоида Bild, в марше приняли участие около 30 тыс. человек. По официальным данным, манифестантов было почти в два раза меньше — 18 тыс. Протестующие начали собираться в субботу с самого утра на улице Унтер-ден-Линден. После этого они прошли маршем через центр Берлина. Финальной точкой должна была стать Улица 17-июня рядом с Брандербургскими воротами. Из-за акции протеста полиции пришлось ограничить движение по улицам Унтер-ден-Линден, Фридрихштрассе. В частности, было перекрыто пространство вокруг Бранденбургских ворот, а также Александерплац и Лейпцигерплац. Российское посольство находится по улице Унтер-ден-Линден. Именно там произошли стычки протестующих с полицией. Там же собравшиеся начали скандировать фамилию российского президента. При этом участники манифестации забрас

In [13]:
trainer.test(model=gen_model, datamodule=gen_dm)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |                                                | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           bleu            │   0.036650583148002625    │
└───────────────────────────┴───────────────────────────┘

[{'bleu': 0.036650583148002625}]

In [22]:
trainer.fit(model=gen_model, datamodule=gen_dm)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type            | Params
------------------------------------------
0 | model | GPT2LMHeadModel | 125 M 
------------------------------------------
125 M     Trainable params
0         Non-trainable params
125 M     Total params
500.926   Total estimated model params size (MB)


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

...

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [23]:
pred_fit = trainer.predict(model=gen_model, datamodule=gen_dm)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |                                             | 0/? [00:00<?, ?it/s]

In [28]:
pred_fit[0][0]

' Берлине прошли массовые акции протеста против антикоронавирусных мер: ношения масок и соблюдения безопасного расстояния в 1,5 м друг от друга. Манифестанты требуют ослабить или отменить ограничения. По данным таблоида Bild, в марше приняли участие около 30 тыс. человек. По официальным данным, манифестантов было почти в два раза меньше — 18 тыс. Протестующие начали собираться в субботу с самого утра на улице Унтер-ден-Линден. После этого они прошли маршем через центр Берлина. Финальной точкой должна была стать Улица 17-июня рядом с Брандербургскими воротами. Из-за акции протеста полиции пришлось ограничить движение по улицам Унтер-ден-Линден, Фридрихштрассе. В частности, было перекрыто пространство вокруг Бранденбургских ворот, а также Александерплац и Лейпцигерплац. Российское посольство находится по улице Унтер-ден-Линден. Именно там произошли стычки протестующих с полицией. Там же собравшиеся начали скандировать фамилию российского президента. При этом участники манифестации забрас

In [25]:
trainer.test(model=gen_model, datamodule=gen_dm)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |                                                | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           bleu            │    0.03614148497581482    │
└───────────────────────────┴───────────────────────────┘

[{'bleu': 0.03614148497581482}]

In [26]:
#### 2 epohs

In [14]:
trainer.fit(model=gen_model, datamodule=gen_dm)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type            | Params
------------------------------------------
0 | model | GPT2LMHeadModel | 125 M 
------------------------------------------
125 M     Trainable params
0         Non-trainable params
125 M     Total params
500.926   Total estimated model params size (MB)


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.


In [15]:
pred_fit = trainer.predict(model=gen_model, datamodule=gen_dm)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |                                             | 0/? [00:00<?, ?it/s]

In [20]:
pred_fit[0][3]

' подобных стрельб нашими военными у границ США?» — указали в заявлении дипмиссии. В Эстонии с 1 по 10 сентября пройдут совместные стрельбы пехотной бригады ВС прибалтийской страны и 41-й бригады полевой артиллерии армии Соединенных Штатов. Это первые боевые стрельбы американской артиллерии за пределами постоянного базирования в Европе. В середине прошлого месяца в НАТО заявили, что альянс не наращивает военное присутствие в Восточной Европе. По словам официального представителя военно-политического блока Оаны Лунгеску, многонациональное присутствие НАТО в восточной части Североатлантического союза не представляет угрозы ни для одного государства. Она также уточнила, что присутствие альянса «носит строго оборонительный, пропорциональный характер и предназначено для предотвращения конфликтов и сохранения мира». Незадолго до этого министр обороны США Марк Эспер, выступая на ежегодном форуме по безопасности, организованном Аспенским институтом, обвинил российские власти в том, что они вын

In [17]:
trainer.test(model=gen_model, datamodule=gen_dm)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |                                                | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           bleu            │    0.03541657701134682    │
└───────────────────────────┴───────────────────────────┘

[{'bleu': 0.03541657701134682}]